In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
import matplotlib.pyplot as plt

In [2]:
with open(os.getcwd()+'\\data\\input.txt', 'r', encoding='utf8') as f:
    text = f.read()

In [2]:
torch.cuda.is_available()

True

In [3]:
chars = sorted(list(set(text)))

vocab_size = len(chars)

print(''.join(chars))
print(vocab_size)

#strategy to tokenize text

#character level mapping

s_to_i = {ch:i for i,ch in enumerate(chars)}
i_to_s = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [s_to_i[c] for c in s]
decode = lambda l: ''.join([i_to_s[c] for c in l])

print(encode("hii there"))
print(decode(encode("hii there")))

#take all of the text and encode them into pytorch tensors

data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65
[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there
torch.Size([1115394]) torch.int64


In [4]:
#splitting into a train and validation split

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [5]:
#input the text sequences into the transformer

#we only work with chunks of the dataset. when we train the model, we sample chunks of the dataset and train on just chunks at a time. chunks have a maximum length: called block_size

block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [6]:
x = train_data[:block_size]

#shift to the right by 1 to get the target
y = train_data[1:block_size+1]

In [7]:
for t in range(block_size):
    context = x[:t+1]
    target = y[t]

    print(f"when the input is {context}, the target will be {target}")

#we want the model to be robust when seeing contexts of multiple lengths, we set it as 8 so that the model has the capability of looking back at least 'block_size' characters of contexts of. 
# When we need to predict the 9th or 10th character, we start to truncate the context to continue onwards

when the input is tensor([18]), the target will be 47
when the input is tensor([18, 47]), the target will be 56
when the input is tensor([18, 47, 56]), the target will be 57
when the input is tensor([18, 47, 56, 57]), the target will be 58
when the input is tensor([18, 47, 56, 57, 58]), the target will be 1
when the input is tensor([18, 47, 56, 57, 58,  1]), the target will be 15
when the input is tensor([18, 47, 56, 57, 58,  1, 15]), the target will be 47
when the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target will be 58


In [9]:
#GLOBALS

batch_size = 4 #This is the value of B
block_size = 8 #This is the value of T
n_embd = 32 #This is the value of C
dropout = 0.4

In [10]:
#getting the batches

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:block_size+1] for i in ix])

    return x, y

xb, yb = get_batch('train')

# print('inputs:')
# print(xb.shape)
# print(xb)
# print('targets:')
# print(yb.shape)
# print(yb)

#xb, yb, of shape (B, T) where B = batch_size, and T = block_size (time sequence)

inputs:
torch.Size([4, 8])
tensor([[27, 26, 32, 17, 31, 10,  0, 21],
        [ 1, 58, 53, 50, 42,  1, 51, 43],
        [61, 53, 59, 50, 42,  1, 53, 52],
        [58, 43, 42,  1, 39, 57,  1, 23]])
targets:
torch.Size([4, 0])
tensor([], size=(4, 0), dtype=torch.int64)


transformer time!

head -> multi-head self attention -> feedforward -> block -> Decoder

special layers:
dropout
layernorm
linear
embedding
pos_embedding

In [11]:
class Head(nn.Module):
    """
    One Head of self-attention 
    """
    def __init__(self, head_size):
        super().__init__()

        self.head_size = head_size
        #initialize the key, query, and value matrices
        self.Wk = nn.Linear(n_embd, head_size)
        self.Wq = nn.Linear(n_embd, head_size)
        self.Wv = nn.Linear(n_embd, head_size)

        #since this is a decoder, we need to initialize the mask as well

        #we register this as a buffer, which still exists as a 'matrix' to use, but we don't compute gradients on this or use in the backward pass
        #model parameters are objects that we use during the forward pass and we update using gradient descent
        #model buffers are objects that we use during computation but do not update

        #both parameters and buffers are saved to the right device when calling .to_device
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        #assume x is of shape (B, T, C)
        B, T, C = x.shape
        K = self.Wk(x) #(B, T, head_size)
        Q = self.Wq(x) #(B, T, head_size)
        V = self.Wv(x) #(B, T, head_size)

        #K.T needs to be of shape (B, C, T), so we swap the -2 and -1 positions
        scores = Q @ K.tranpose(-2, -1) * 1/(self.head_size)**2 #(B, T, T)
        masked_scores = scores.masked_fill(self.tril[:T, :T] == 0, float('-inf')) #(B, T, T)
        attention_scores = F.softmax(masked_scores, dim = -1) #applying softmax along the rows (B, T, T)
        attention_scores = self.dropout(attention_scores) #(B, T, T)
        
        out = attention_scores @ V #(B, T, head_size)
        
        return out #(B, T, head_size)


In [12]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)]) #(B, T, n_heads*head_size)
        self.proj = nn.Linear(n_heads * head_size, n_embd) #paper specifies a final linear layer
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        #assume input x is of size (B, T, C)

        #Each Head returns a output of size (B, T, head_size), we concatenate along the final dimension so that our variable 'out' is now (B, T, n_heads*head_size)
        out = torch.cat([h(x) for h in self.heads], dim = -1)
        out = self.proj(out) #(B, T, C)
        out = self.dropout(out)

        return out

In [13]:
class FeedForward(nn.Module):
    def __init__(self):
        #input and output will be of size (B, T, C)
        self.ff1 = nn.Linear(n_embd, 4*n_embd)
        self.ff2 = nn.Linear(4*n_embd, n_embd)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        #assume x is of shape (B, T, C)
        x = self.ff1(x)
        x = self.relu(x)
        x = self.ff2(x)
        x = dropout(x)

        return x

In [14]:
class Block(nn.Module):
    #implementaion of one transformer block
    def __init__(self, n_head):
        super().__init__()
        self.head_size = n_embd / n_head

        self.sa = MultiHeadAttention(n_head, self.head_size)
        self.ffw = FeedForward(n_embd)
        self.layernorm1 = nn.LayerNorm(n_embd)
        self.layernorm2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        #assume input of x is size (B, T, C) where x is the sum of the embedded input + positional_encoding
        x = self.layernorm1(x) #(B, T, C)
        x = x + self.sa(x) #(B, T, C)
        x = self.layernorm2(x) #(B, T, C)
        x = x + self.ffw(x) #(B, T, C)

        return x
